In [1]:
import pymysql
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols


import statsmodels.stats.multicomp

In [2]:
con = pymysql.connect(host="localhost", user="gabic", passwd="1234", db="doenca_renal_cronica")
con.select_db('doenca_renal_cronica')
cursor = con.cursor()  
cursor.connection.autocommit(True)

In [3]:
#criando os dataframes por tabela 
dados_consulta = pd.read_sql('SELECT * FROM doenca_renal_cronica.dados_consulta',con=con)
cidade = pd.read_sql('SELECT * FROM doenca_renal_cronica.cidade',con=con)
ubs = pd.read_sql('SELECT * FROM doenca_renal_cronica.ubs',con=con)
aval_funcao_renal = pd.read_sql('SELECT * FROM doenca_renal_cronica.aval_funcao_renal',con=con)
consulta_has = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_has',con=con)
consulta_drc = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_drc',con=con)
consulta_dm = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_dm',con=con)
calculo_tfg = pd.read_sql('SELECT * FROM doenca_renal_cronica.calculo_tfg',con=con)
preparacao_trs = pd.read_sql('SELECT * FROM doenca_renal_cronica.preparacao_trs',con=con)
raca = pd.read_sql('SELECT * FROM doenca_renal_cronica.raca',con=con)
instrucao = pd.read_sql('SELECT * FROM doenca_renal_cronica.instrucao',con=con)
paciente = pd.read_sql('SELECT * FROM doenca_renal_cronica.paciente',con=con)
exame_fisico = pd.read_sql('SELECT * FROM doenca_renal_cronica.exame_fisico',con=con)
avaliacao_tratamento = pd.read_sql('SELECT * FROM doenca_renal_cronica.avaliacao_tratamento',con=con)
medicamento = pd.read_sql('SELECT * FROM doenca_renal_cronica.medicamento',con=con)
complicacao_funcao_renal = pd.read_sql('SELECT * FROM doenca_renal_cronica.complicacao_funcao_renal',con=con)

#dicionario com todas tabelas
dic = {'dados_consulta':dados_consulta,'cidade':cidade,'ubs':ubs,'aval_funcao_renal':aval_funcao_renal,
       'consulta_has':consulta_has,'consulta_drc':consulta_drc,'consulta_dm':consulta_dm,'calculo_tfg':calculo_tfg,
       'preparacao_trs':preparacao_trs,'raca':raca,'instrucao':instrucao,'paciente':paciente,'exame_fisico':exame_fisico,
       'avaliacao_tratamento':avaliacao_tratamento,'medicamento':medicamento,'complicacao_funcao_renal':complicacao_funcao_renal
      }

In [4]:
#separando apenas as colunas com números

df_num_dados_consulta = dados_consulta.select_dtypes(include = ['float64', 'int64'])
df_num_cidade = cidade.select_dtypes(include = ['float64', 'int64'])
df_num_ubs = ubs.select_dtypes(include = ['float64', 'int64'])
df_num_aval_funcao_renal = aval_funcao_renal.select_dtypes(include = ['float64', 'int64'])
df_num_consulta_has = consulta_has.select_dtypes(include = ['float64', 'int64'])
df_num_consulta_drc = consulta_drc.select_dtypes(include = ['float64', 'int64'])
df_num_consulta_dm = consulta_dm.select_dtypes(include = ['float64', 'int64'])
df_num_calculo_tfg = calculo_tfg.select_dtypes(include = ['float64', 'int64'])
df_num_preparacao_trs = preparacao_trs.select_dtypes(include = ['float64', 'int64'])
df_num_raca = raca.select_dtypes(include = ['float64', 'int64'])
df_num_instrucao = instrucao.select_dtypes(include = ['float64', 'int64'])
df_num_paciente = paciente.select_dtypes(include = ['float64', 'int64'])
df_num_exame_fisico = exame_fisico.select_dtypes(include = ['float64', 'int64'])
df_num_avaliacao_tratamento = avaliacao_tratamento.select_dtypes(include = ['float64', 'int64'])
df_num_medicamento = medicamento.select_dtypes(include = ['float64', 'int64'])
df_num_complicacao_funcao_renal = complicacao_funcao_renal.select_dtypes(include = ['float64', 'int64'])

dic_num = {'dados_consulta':df_num_dados_consulta,'cidade':df_num_cidade,'ubs':df_num_ubs,'afr':df_num_aval_funcao_renal,
       'has':df_num_consulta_has,'drc':df_num_consulta_drc,'dm':df_num_consulta_dm,'calculo_tfg':df_num_calculo_tfg,
       'preparacao_trs':df_num_preparacao_trs,'raca':df_num_raca,'instrucao':df_num_instrucao,'paciente':df_num_paciente,'exame_fisico':df_num_exame_fisico,
       'aval_tratamento':df_num_avaliacao_tratamento,'medicamento':df_num_medicamento,'cfr':df_num_complicacao_funcao_renal
      }

for i in dic_num:
    dic_num[i]=dic_num[i].drop(columns=['id_'+i])

In [5]:
##Retirando os dados inexistentes, ou seja, ==0
##criar um dicionario com as colunas sem valor nulo e com os nomes originais do df
##depois transformar este em um dataFrame para poder utilizar os modelos estatisticos
# semZero_tri = dic_num['cfr']['triglicerides_inicial'].drop(dic_num['cfr']!= 0.0, columns = )  
# semZero_tri
triglicerides_inicial_snulo = dic_num['cfr']['triglicerides_inicial']
triglicerides_inicial_snulo = triglicerides_inicial_snulo [triglicerides_inicial_snulo.values != 0.0]

glicemia_jejum_inicial_snulo = dic_num['cfr']['glicemia_jejum_inicial']
glicemia_jejum_inicial_snulo = glicemia_jejum_inicial_snulo[glicemia_jejum_inicial_snulo != 0.0]

colest_total_inicial_snulo = dic_num['cfr']['colest_total_inicial']
colest_total_inicial_snulo = colest_total_inicial_snulo[colest_total_inicial_snulo != 0.0]

triglicerides_final_snulo = dic_num['cfr']['triglicerides_final']
triglicerides_final_snulo = triglicerides_final_snulo [triglicerides_final_snulo!= 0.0]

glicemia_jejum_final_snulo = dic_num['cfr']['glicemia_jejum_final']
glicemia_jejum_final_snulo = glicemia_jejum_final_snulo[glicemia_jejum_final_snulo != 0.0]

colest_total_final_snulo = dic_num['cfr']['colest_total_final']
colest_total_final_snulo = colest_total_final_snulo[colest_total_final_snulo != 0.0]

dict_snulo = {'triglicerides_inicial': triglicerides_inicial_snulo, 
             'glicemia_jejum_inicial':glicemia_jejum_inicial_snulo,
             'colest_total_inicial':colest_total_inicial_snulo,
             'triglicerides_final': triglicerides_final_snulo, 
             'glicemia_jejum_final':glicemia_jejum_final_snulo,
             'colest_total_final':colest_total_final_snulo          
            }
df_snulo = pd.DataFrame(dict_snulo)

# Post-hoc Testing

In [6]:
mc = statsmodels.stats.multicomp.MultiComparison(df_snulo ['triglicerides_inicial'], df_snulo ['triglicerides_final'])
mc_results = mc.tukeyhsd()
print(mc_results)

KeyboardInterrupt: 